✅ AutoAWQ → Loads the quantized model efficiently
✅ FastAPI → Creates the API
✅ Uvicorn → Runs the API
✅ Pyngrok → Exposes the API for external access
✅ Transformers/Accelerate → Optimized inference

In [3]:
!pip install transformers torch fastapi uvicorn huggingface_hub pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [20]:
!ngrok authtoken YOUR ngrok Token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
from huggingface_hub import login

# Replace with your actual token
hf_token = "Your HF Token"

# Login to Hugging Face
login(token=hf_token)


In [25]:


# Import libraries
import torch
from fastapi import FastAPI
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel
import uvicorn
import threading
from pyngrok import ngrok
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Define FastAPI app
app = FastAPI()

# # Load model (Meditron-7B)
# MODEL_NAME = "EPFL-LLM/meditron-7b"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16,
#     device_map="auto"  # Use GPU if available
# )

# # Load ClinicalBERT tokenizer & model
# MODEL_NAME = "medicalai/ClinicalBERT"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

# # Move model to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
# Request format
class QueryRequest(BaseModel):
    query: str

@app.post("/query")
def get_response(request: QueryRequest):
    input_text = request.query
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150)
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return {"response": response_text}

# Function to run FastAPI server
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok tunnel
public_url = ngrok.connect(8000).public_url
print(f"Public API URL: {public_url}")

# Run FastAPI server in a separate thread
thread = threading.Thread(target=run_api)
thread.start()


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Public API URL: https://e553-35-204-188-17.ngrok-free.app
